In [1]:
# Basic packages
import numpy as np
#import pandas as pd

In [2]:
#total_reactions = 0
#detector_area = 6.7 * 10**-6 #m^2 (2.4 mm * 2.8 mm)
#rs = [2*10**-2,2*10**-2,2*10**-2,2*10**-2]
#thetas = 
#phis

In [2]:
def gamma_calc(m1, m2, m3, m4, en1, Q):
    return np.sqrt(m1*m3/(m2*m4)*en1/(en1+Q*(1+m1/m2)))

def en3_calc(m1, m3, m4, theta_L, Q, en1):

    theta_L = np.radians(theta_L)
    v = np.sqrt(m1*m3*en1)/(m3+m4)*np.cos(theta_L)
    w = (m4*Q+en1*(m4-m1))/(m3+m4)
    
    #Test to see if the energies are real.
    e3_p = v**2+w
    e3_n = v**2+w

    e3_p[e3_p < 0] = np.nan
    e3_n[e3_n < 0] = np.nan

    e3_p = v + np.sqrt(e3_p)
    e3_n = v - np.sqrt(e3_n)

    #Test to see if energies are negative.
    e3_p[e3_p < 0] = np.nan
    e3_n[e3_n < 0] = np.nan

    e3 = np.full(len(theta_L), np.nan)

    w_e3_p = np.where(~np.isnan(e3_p) == True)
    w_e3_n = np.where(~np.isnan(e3_n) == True)
    w_both = np.intersect1d(w_e3_p, w_e3_n)

    mixed = np.array([e3_p[w_both], e3_n[w_both]])

    decide = np.random.randint(0,2,len(w_both))

    e3[~np.isnan(e3_p)] = e3_p[~np.isnan(e3_p)]
    e3[~np.isnan(e3_n)] = e3_n[~np.isnan(e3_n)]

    e3[w_both] = np.choose(decide, mixed)

    return e3**2


def theta_LtoC(theta_C, gamma):
    theta_L = np.degrees(np.arctan(np.sin(np.radians(theta_C))/(gamma+np.cos(np.radians(theta_C)))))

    #Check for 0 deg and 180 deg same in C-corrdinates
    theta_L[theta_C == 0] = 0
    theta_L[theta_C == 180] = 180

    #If negative angle add 180
    theta_L[theta_L < 0] = theta_L[theta_L < 0] + 180

    return theta_L

def en4_calc(en1, en3, Q):
    return en1 + Q - en3

def phi_L_calc(theta_L, m3, en3, m4, en4):
    constant = np.sqrt(2*m3*en3)/np.sqrt(2*m4*en4)
    phi_L = np.degrees(np.arcsin(constant*np.sin(np.radians(theta_L))))
    return phi_L

def gamma_disintegration(mi, m1, m2, eni, Q):
    return m1/np.sqrt(m1*m2*Q/(m1+m2))*np.sqrt(eni/mi)



In [3]:
mp=1.00727646 #mass of proton
mB11=11.0093052 #mass of B11
mAlpha=4.002602 #mass of alpha
mBe8=8.00530510 #mass of Be8
QBe8=8.586 #Q value of Be8
QBe8s=5.65 #Q value of excited Be8
Qa=0.092 #Q value for alpha_12 from non-excited Be8
Qas=3.028 #Q value for alpha_12 from excited Be8

total_reactions = 1000
theta_C_array1 = np.random.uniform(0,180,total_reactions)
#azimuthal_array1 = np.random.uniform(0,360,total_reactions)
#azimuthal_array2 = np.random.uniform(0,360,total_reactions)
theta_array2 = np.random.uniform(0,180,total_reactions)

Q1 = np.random.choice([QBe8,QBe8s],total_reactions,p=[0.01,0.99])

In [13]:
#First reaction calculations
enp = 0.66

gamma1 = gamma_calc(mp, mB11, mAlpha, mAlpha, enp, Q1)

theta_L1 = theta_LtoC(theta_C_array1, gamma1)

en_alpha_01 = en3_calc(mp, mAlpha, mBe8, theta_L1, Q1, enp)

#en_alpha_01 = np.concatenate((en_alpha_01_p, en_alpha_01_n))
#theta_L1 = np.concatenate((theta_L1, theta_L1))
#Q1 = np.concatenate((Q1, Q1))

#en_Be8 = en4_calc(enp, en_alpha_01, Q1)

#phi_L1 = phi_L_calc(theta_L1, mAlpha, en_alpha_01, mBe8, en_Be8)

#w_nonnan1 =np.where(~np.isnan(en_alpha_01))

#en_alpha_01 = en_alpha_01[w_nonnan1]
#theta_L1 = theta_L1[w_nonnan1]
#Q1 = Q1[w_nonnan1]
#en_Be8 = en_Be8[w_nonnan1]
#phi_L1 = phi_L1[w_nonnan1]

# #Second reaction calculations
# theta_C_array2 = np.random.uniform(0,180,total_reactions)

# Q2 = np.full((len(Q1)),Qas)
# Q2[Q1 == QBe8] = Qa

# gamma2 = gamma_disintegration(mBe8, mAlpha, mAlpha, en_Be8, Q2)
# theta_L2 = theta_LtoC(theta_C_array2, gamma2)

# en_alpha_11_p, en_alpha_11_n = en3_calc(mBe8, mAlpha, mAlpha, theta_L2, Q2, en_Be8)

# en_alpha_11 = np.concatenate((en_alpha_11_p, en_alpha_11_n))
# theta_L2 = np.concatenate((theta_L2, theta_L2))
# en_Be8 = np.concatenate((en_Be8, en_Be8))
# Q2 = np.concatenate((Q2, Q2))

# en_alpha_12 = en4_calc(en_Be8, en_alpha_11, Q2)

# phi_L2 = phi_L_calc(theta_L2, mAlpha, en_alpha_11, mAlpha, en_alpha_12)

# w_nonnan2 =np.where(~np.isnan(en_alpha_11))

# en_alpha_11 = en_alpha_11[w_nonnan2]
# theta_L2 = theta_L2[w_nonnan2]
# en_Be8 = en_Be8[w_nonnan2]
# Q2 = Q2[w_nonnan2]
# en_alpha_12 = en_alpha_12[w_nonnan2]
# phi_L2 = phi_L2[w_nonnan2]


In [13]:
#phi_L_calc(theta_L1,mAlpha, en_alpha_01, mBe8, en_Be8)

#phi_L_calc(theta_L, m3, en3, m4, en4):
np.sqrt(2*mAlpha*en_alpha_01)#/np.sqrt(2*mBe8*en_Be8)

TypeError: can't multiply sequence by non-int of type 'float'

In [7]:
en3_calc(mp, mAlpha, mBe8, theta_L1, Q1, enp)[0]

array([3.65417283, 4.5743486 , 3.64470708, 4.74290866, 3.63383789,
       3.9120672 , 4.33364147, 4.41399698, 3.76821695, 4.06054331,
       4.01643504, 4.41770317, 4.07561168, 3.69328735, 4.69067966,
       4.4869864 , 4.65828651, 4.64162112, 3.79126421, 4.61939086,
       4.5237515 , 3.96500453, 4.74295047, 4.72461851, 4.37492758,
       3.75018157, 4.15268101, 4.74261369, 4.48365051, 4.74300851,
       4.40805212, 4.73232065, 4.64604935, 3.89520858, 3.66293268,
       4.25749399, 3.69922622, 4.6460822 , 3.63419784, 4.67587251,
       4.36909248, 4.64816684, 4.47830986, 4.63640087, 3.64459666,
       4.23134065, 4.16451396, 4.02214632, 4.38596479, 4.74216978,
       4.73897097, 4.1740576 , 3.85833329, 3.79667936, 3.67216353,
       4.72707856, 4.68447642, 4.02918979, 3.75332169, 3.98845179,
       4.54876077, 3.79075752, 4.6734028 , 4.68348345, 4.53622832,
       4.74302954, 3.75091626, 4.53626549, 4.33918222, 4.50806109,
       4.59005009, 4.1136014 , 4.00817386, 3.7423575 , 4.67363

In [26]:
print(len(en_alpha_01))
print(len(theta_L1))
print(len(phi_L1))

print(len(en_alpha_11))
print(len(en_alpha_12))


1000
1000
1000
1009
1009


In [11]:
phi_L1

array([80.09157902, 50.75267202, 73.67652652, ...,         nan,
               nan,         nan])

In [24]:
test = np.array([1,2,np.nan,np.nan,5,7,8,9])
test2 = np.array([np.nan,np.nan,3,4,6,1,2,3])
test3 = np.full(len(test), np.nan)

w_test = np.where(~np.isnan(test) == True)
w_test2 = np.where(~np.isnan(test2) == True)
w_both = np.intersect1d(w_test, w_test2)

mixed = np.array([test[w_both], test2[w_both]])

decide = np.random.randint(0,2,len(w_both))

test3[~np.isnan(test)] = test[~np.isnan(test)]
test3[~np.isnan(test2)] = test2[~np.isnan(test2)]

test3[w_both] = np.choose(decide, mixed)

In [25]:
test3


array([1., 2., 3., 4., 5., 1., 8., 9.])

In [110]:
decide

array([0, 1, 1, 0])

In [107]:
len(np.where(np.random.randint(0,2,4)==1)[0])

3

In [108]:
np.random.randint(0,2,4)

array([1, 0, 1, 0])

In [8]:
#def en3_calc(m1, m3, m4, theta_L, Q, en1):

m1= mp
m3= mAlpha
m4= mBe8
theta_L = theta_L1
Q = Q1
en1 = enp

theta_L = np.radians(theta_L)
v = np.sqrt(m1*m3*en1)/(m3+m4)*np.cos(theta_L)
w = (m4*Q+en1*(m4-m1))/(m3+m4)

#Test to see if the energies are real.
e3_p = v**2+w
e3_n = v**2+w

e3_p[e3_p < 0] = np.nan
e3_n[e3_n < 0] = np.nan

e3_p = v + np.sqrt(e3_p)
e3_n = v - np.sqrt(e3_n)

#Test to see if energies are negative.
e3_p[e3_p < 0] = np.nan
e3_n[e3_n < 0] = np.nan

e3 = np.full(len(theta_L), np.nan)

w_e3_p = np.where(~np.isnan(e3_p) == True)
w_e3_n = np.where(~np.isnan(e3_n) == True)
w_both = np.intersect1d(w_e3_p, w_e3_n)

mixed = np.array([e3_p[w_both], e3_n[w_both]])

decide = np.random.randint(0,2,len(w_both))

e3[~np.isnan(e3_p)] = e3_p[~np.isnan(e3_p)]
e3[~np.isnan(e3_n)] = e3_n[~np.isnan(e3_n)]

e3[w_both] = np.choose(decide, mixed)

#return e3**2, 

In [9]:
e3

array([1.91158908, 2.13877268, 1.9091116 , 2.177822  , 1.90626281,
       1.97789464, 2.08174001, 2.10095145, 1.94118957, 2.01507898,
       2.00410455, 2.10183329, 2.01881442, 1.92179274, 2.1657977 ,
       2.11825079, 2.1583064 , 2.15444218, 1.9471169 , 2.14927682,
       2.12691126, 1.99123191, 2.1778316 , 2.17361876, 2.09163275,
       1.93653855, 2.0378128 , 2.17775428, 2.11746322, 2.17784492,
       2.09953617, 2.17538977, 2.15546964, 1.97362828, 1.91387896,
       2.06336957, 1.92333726, 2.15547726, 1.90635722, 2.16237659,
       2.09023742, 2.15596077, 2.11620175, 2.15323033, 1.90908268,
       2.05702228, 2.04071408, 2.00552894, 2.09426951, 2.17765236,
       2.17691777, 2.04305105, 1.96426406, 1.94850696, 1.916289  ,
       2.17418457, 2.16436513, 2.00728418, 1.93734914, 1.99711086,
       2.1327824 , 1.94698678, 2.16180545, 2.16413573, 2.12984232,
       2.17784975, 1.93672824, 2.12985105, 2.08307038, 2.12321951,
       2.14244022, 2.02820152, 2.00204242, 1.93451738, 2.16185